In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [51.8 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,42

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [9]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)


In [10]:
# 2. Create a temporary view of the DataFrame.
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [12]:
df.select('price').show()

+------+
| price|
+------+
|936923|
|379628|
|417866|
|239895|
|424418|
|218712|
|419199|
|323956|
|181925|
|258710|
|167864|
|337527|
|229896|
|210247|
|398667|
|437958|
|437375|
|288650|
|308313|
|177541|
+------+
only showing top 20 rows



In [13]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import avg, round
avg_price_df = df.filter(df.bedrooms == 4).groupBy("date_built").agg(round(avg("price"), 2).alias("average_price"))
avg_price_df.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    296050.24|
|      2012|    298233.42|
|      2017|    296576.69|
|      2014|    299073.89|
|      2013|    299999.39|
|      2011|     302141.9|
|      2015|    307908.86|
|      2010|    296800.75|
+----------+-------------+



In [15]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

avg_price_3_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3)).groupBy("date_built").agg(round(avg("price"), 2).alias("average_price_3bd_3bth"))
avg_price_3_df.show()

+----------+----------------------+
|date_built|average_price_3bd_3bth|
+----------+----------------------+
|      2016|             290555.07|
|      2012|             293683.19|
|      2017|             292676.79|
|      2014|             290852.27|
|      2013|             295962.27|
|      2011|             291117.47|
|      2015|              288770.3|
|      2010|             292859.62|
+----------+----------------------+



In [16]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_grter_2000_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3) & (df.floors == 2) & (df.sqft_living >= 2000)).groupBy("date_built").agg(round(avg("price"), 2).alias("average_price_3bd_3bth_2floors_2000sqft"))
avg_price_grter_2000_df.show()

+----------+---------------------------------------+
|date_built|average_price_3bd_3bth_2floors_2000sqft|
+----------+---------------------------------------+
|      2016|                               293965.1|
|      2012|                              307539.97|
|      2017|                              280317.58|
|      2014|                              298264.72|
|      2013|                              303676.79|
|      2011|                              276553.81|
|      2015|                              297609.97|
|      2010|                              285010.22|
+----------+---------------------------------------+



In [24]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_per_view_df = df.filter(df.price >= 350000).groupBy("view").agg(round(avg("price"), 2).alias("average_view"))
avg_per_view_df.orderBy(avg_per_view_df["average_view"].desc()).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|average_view|
+----+------------+
|  91|  1137372.73|
|  97|  1129040.15|
|  84|  1117233.13|
|  75|  1114042.94|
|  89|  1107839.15|
|  78|  1080649.37|
|  77|  1076205.56|
|  87|   1072285.2|
|  86|  1070444.25|
|  82|   1063498.0|
|  90|  1062654.16|
|  99|  1061201.42|
|  76|  1058802.78|
|  85|  1056336.74|
|  95|   1054325.6|
|  98|  1053739.33|
|  81|  1053472.79|
|  83|  1033965.93|
|  94|   1033536.2|
|  88|  1031719.35|
+----+------------+
only showing top 20 rows

--- 0.7000911235809326 seconds ---


In [38]:
# 7. Cache the the temporary table home_sales.
cache_df = df.createOrReplaceTempView('home_sales')
spark.catalog.cacheTable('home_sales')

In [39]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [45]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

average_price_by_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    GROUP BY view
    HAVING ROUND(AVG(price), 2) >= 350000
""")

average_price_by_view_cached.orderBy(average_price_by_view_cached["average_price"].desc()).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 0.5343155860900879 seconds ---


In [46]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [48]:
# 11. Read the parquet formatted data.
p_df_p = spark.read.parquet('home_sales')
p_df_p.show()

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a2-86f3-46b...|2022-02-18|419543|       3|        2|       1642|   12826|

In [49]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales')

In [51]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

average_price_by_view_uncached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM p_home_sales
    GROUP BY view
    HAVING ROUND(AVG(price), 2) >= 350000
""")

average_price_by_view_uncached.orderBy(average_price_by_view_uncached["average_price"].desc()).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  91|   1137372.73|
|  97|   1129040.15|
|  84|   1117233.13|
|  75|   1114042.94|
|  89|   1107839.15|
|  78|   1080649.37|
|  77|   1076205.56|
|  87|    1072285.2|
|  86|   1070444.25|
|  82|    1063498.0|
|  90|   1062654.16|
|  99|   1061201.42|
|  76|   1058802.78|
|  85|   1056336.74|
|  95|    1054325.6|
|  98|   1053739.33|
|  81|   1053472.79|
|  83|   1033965.93|
|  94|    1033536.2|
|  88|   1031719.35|
+----+-------------+
only showing top 20 rows

--- 1.0539474487304688 seconds ---


In [52]:
# 14. Uncache the home_sales temporary table.
uncache_df = spark.sql("uncache table home_sales")

In [54]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False